In [1]:
# Install Spark

import os
# Find the latest version of spark 3.2.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.2/spark-3.2.2-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"
os.environ["SPARK_CLASSPATH"] = "/content/postgresql-42.2.16.jar"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.postgresql:postgresql:42.1.1 pyspark-shell'

# Start a SparkSession
import findspark
findspark.init()

# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
--2022-10-24 19:23:32--  https://jdbc.postgresql.or

In [2]:
# start a Spark session with an additional option that adds the driver to Spark
from pyspark.conf import SparkConf

conf = SparkConf()
conf.set("spark.jars", "/content/postgresql-42.2.16.jar")
conf.set("spark.driver.extraClassPath", "/content/postgresql-42.2.16.jar")
conf.set("spark.executor.extraClassPath", "/content/postgresql-42.2.16.jar")

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()


**Load Data**

In [3]:
# Read in data 
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Toys_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
vine_reviews_clean_df = df.dropna()

In [6]:
# Create the vine_table DataFrame
vine_df = vine_reviews_clean_df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RDIJS7QYB6XNR|          5|            0|          0|   N|                Y|
|R36ED1U38IELG8|          5|            0|          0|   N|                Y|
| R1UE3RPRGCOLD|          2|            1|          1|   N|                Y|
|R298788GS6I901|          5|            0|          0|   N|                Y|
|  RNX4EXOBBPN5|          1|            1|          1|   N|                Y|
|R3BPETL222LMIM|          5|            0|          0|   N|                Y|
|R3SORMPJZO3F2J|          3|            2|          2|   N|                Y|
|R2RDOJQ0WBZCF6|          5|            0|          0|   N|                Y|
|R2B8VBEPB4YEZ7|          5|            0|          0|   N|                Y|
|R1CB783I7B0U52|          1|            0|          1|   N|     

**Filter Data**

In [7]:
# Filtered for more than 20 total votes
vine_filtered_votes_df = vine_df.filter("total_votes>=20")
vine_filtered_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| ROP6ITXO8K5V2|          5|           23|         27|   N|                Y|
|R3ND1LVU7AXCVF|          1|           21|         21|   N|                Y|
| R9I5FOLKU99RY|          5|           19|         20|   Y|                N|
|R1QS8AOD6HX3ED|          4|           59|         81|   N|                N|
|R3ED60RC69CCQ6|          5|           22|         23|   Y|                N|
|R2MDI1TP46VSYD|          3|            9|         21|   N|                Y|
|R2JM687C525WR9|          3|           33|         33|   Y|                N|
|R2LWX4TZ67FWPT|          4|           50|         50|   N|                N|
|R29IYHPYD14AGI|          3|           84|         84|   N|                Y|
|R1F2I723WRK5QV|          5|           20|         20|   N|     

In [8]:
# Filtered for more than 20 total votes, and more than 50% helpful
vine_helpful_by_total_df = vine_filtered_votes_df.filter("(helpful_votes/total_votes)>=0.5")
vine_helpful_by_total_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| ROP6ITXO8K5V2|          5|           23|         27|   N|                Y|
|R3ND1LVU7AXCVF|          1|           21|         21|   N|                Y|
| R9I5FOLKU99RY|          5|           19|         20|   Y|                N|
|R1QS8AOD6HX3ED|          4|           59|         81|   N|                N|
|R3ED60RC69CCQ6|          5|           22|         23|   Y|                N|
|R2JM687C525WR9|          3|           33|         33|   Y|                N|
|R2LWX4TZ67FWPT|          4|           50|         50|   N|                N|
|R29IYHPYD14AGI|          3|           84|         84|   N|                Y|
|R1F2I723WRK5QV|          5|           20|         20|   N|                Y|
| RRO8C1IVZMD6H|          5|           22|         22|   N|     

In [9]:
# Filtered for paid reviews, more than 50% helpful
vine_paid_helpful_df = vine_helpful_by_total_df.filter("vine == 'Y'")
vine_paid_helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R9I5FOLKU99RY|          5|           19|         20|   Y|                N|
|R3ED60RC69CCQ6|          5|           22|         23|   Y|                N|
|R2JM687C525WR9|          3|           33|         33|   Y|                N|
| RUB4AUWGHG16G|          4|           25|         27|   Y|                N|
|R1S3IDYJUA6V2G|          5|           28|         32|   Y|                N|
| RZFLN8FD5Y2M6|          4|           25|         28|   Y|                N|
|R1V8NDXD9SETO5|          3|          320|        352|   Y|                N|
|R2H4VEFWHP50M3|          5|           26|         36|   Y|                N|
|R3DP6Y0A8WXUOV|          5|           21|         21|   Y|                N|
| R49L85GZIKH4D|          3|           33|         36|   Y|     

In [10]:
# Filtered for non-paid reviews, more than 50% helpful
vine_unpaid_helpful_df = vine_helpful_by_total_df.filter("vine == 'N'")
vine_unpaid_helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| ROP6ITXO8K5V2|          5|           23|         27|   N|                Y|
|R3ND1LVU7AXCVF|          1|           21|         21|   N|                Y|
|R1QS8AOD6HX3ED|          4|           59|         81|   N|                N|
|R2LWX4TZ67FWPT|          4|           50|         50|   N|                N|
|R29IYHPYD14AGI|          3|           84|         84|   N|                Y|
|R1F2I723WRK5QV|          5|           20|         20|   N|                Y|
| RRO8C1IVZMD6H|          5|           22|         22|   N|                N|
|R2GHOWIWDVLN18|          1|           28|         36|   N|                Y|
|R30PJ9XUHFTY5D|          5|           42|         43|   N|                N|
|R2F7D9N3SSQC20|          4|           32|         34|   N|     

**Calculations**

In [22]:
# Total number of reviews
total_reviews_ct = vine_helpful_by_total_df.count()
total_reviews_ct

63270

In [23]:
# Total paid reviews
total_paid_reviews_df = vine_helpful_by_total_df.filter("vine =='Y'")
total_paid_reviews_ct = total_paid_reviews_df.count()
total_paid_reviews_ct

1266

In [24]:
# Total unpaid reviews
total_unpaid_reviews_df = vine_helpful_by_total_df.filter("vine =='N'")
total_unpaid_reviews_ct = total_unpaid_reviews_df.count()
total_unpaid_reviews_ct

62004

In [25]:
# Total five-star reviews
total_five_star_reviews_df = vine_helpful_by_total_df.filter("star_rating == 5")
total_five_star_reviews_ct = total_five_star_reviews_df.count()
total_five_star_reviews_ct

30397

In [26]:
# Total five-star paid reviews
five_star_paid_reviews_df = total_paid_reviews_df.filter("star_rating == 5")
five_star_paid_reviews_ct = five_star_paid_reviews_df.count()
five_star_paid_reviews_ct

432

In [27]:
# Total five-star unpaid reviews
five_star_unpaid_reviews_df = total_unpaid_reviews_df.filter("star_rating == 5")
five_star_unpaid_reviews_ct = five_star_unpaid_reviews_df.count()
five_star_unpaid_reviews_ct

29965

In [28]:
# Five-star paid reviews as percent of total paid reviews
paid_five_star_per_total_paid = (five_star_paid_reviews_ct/total_paid_reviews_ct)*100
round(paid_five_star_per_total_paid, 3)

34.123

In [29]:
# Paid reviews as percent of total five-star reviews
paid_five_star_per_five_star_total = (five_star_paid_reviews_ct/total_five_star_reviews_ct)*100
round(paid_five_star_per_five_star_total, 3)

1.421

In [30]:
# Five-star unpaid reviews as percent of total unpaid reviews
unpaid_five_star_per_total_unpaid = (five_star_unpaid_reviews_ct/total_unpaid_reviews_ct)*100
round(unpaid_five_star_per_total_unpaid, 3)

48.328

In [31]:
# Five-star unpaid reviews as percent of total five-star reviews
unpaid_five_star_per_five_star_total = (five_star_unpaid_reviews_ct/total_five_star_reviews_ct)*100
round(unpaid_five_star_per_five_star_total, 3)

98.579